# Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [2]:
digits=load_digits()
#digits

{'data': array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'target': array([0, 1, 2, ..., 8, 9, 8]),
 'frame': None,
 'feature_names': ['pixel_0_0',
  'pixel_0_1',
  'pixel_0_2',
  'pixel_0_3',
  'pixel_0_4',
  'pixel_0_5',
  'pixel_0_6',
  'pixel_0_7',
  'pixel_1_0',
  'pixel_1_1',
  'pixel_1_2',
  'pixel_1_3',
  'pixel_1_4',
  'pixel_1_5',
  'pixel_1_6',
  'pixel_1_7',
  'pixel_2_0',
  'pixel_2_1',
  'pixel_2_2',
  'pixel_2_3',
  'pixel_2_4',
  'pixel_2_5',
  'pixel_2_6',
  'pixel_2_7',
  'pixel_3_0',
  'pixel_3_1',
  'pixel_3_2',
  'pixel_3_3',
  'pixel_3_4',
  'pixel_3_5',
  'pixel_3_6',
  'pixel_3_7',
  'pixel_4_0',
  'pixel_4_1',
  'pixel_4_2',
  'pixel_4_3',
  'pixel_4_4',
  'pixel_4_5',
  'pixel_4_6',
  'pixel_4_7',
  'pixel_5_0',
  'pixel_5_1',
 

In [3]:
dir(digits)

['DESCR', 'data', 'feature_names', 'frame', 'images', 'target', 'target_names']

In [4]:
digits.data

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [5]:
data = digits.images.reshape((len(digits.images), -1))
data

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [6]:
data = np.pad(data, ((0, 0), (1, 1)), mode='constant')
data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 9., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [7]:
X_train, X_temp, y_train, y_temp = train_test_split(data, digits.target, test_size=0.2, stratify=digits.target)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)


In [8]:
X_train.shape

(1437, 66)

In [9]:
X_test.shape

(180, 66)

In [10]:
X_temp.shape

(360, 66)

# Implement perceptron and evaluate its performance

In [49]:
class Perceptron:
    def __init__(self, learning_rate, epochs):
        self.lr = learning_rate
        self.epochs = epochs
        self.weights = np.zeros(X_train.shape[1])

    def fit(self, X, y):
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                x = X[i]
                y_pred = np.dot(self.weights, x)
                y_error = y[i] - y_pred
                self.weights += self.lr * y_error * x

    def predict(self, X):
        y_pred = np.dot(self.weights, X.T)
        return np.sign(y_pred)



Training Perceptron with linear activation:
Epoch 1/30 - Training Loss: 2.3234
Epoch 2/30 - Training Loss: 2.3234
Epoch 3/30 - Training Loss: 2.3234
Epoch 4/30 - Training Loss: 2.3234
Epoch 5/30 - Training Loss: 2.3234
Epoch 6/30 - Training Loss: 2.3234
Epoch 7/30 - Training Loss: 2.3234
Epoch 8/30 - Training Loss: 2.3234
Epoch 9/30 - Training Loss: 2.3234
Epoch 10/30 - Training Loss: 2.3234
Epoch 11/30 - Training Loss: 2.3234
Epoch 12/30 - Training Loss: 2.3234
Epoch 13/30 - Training Loss: 2.3234
Epoch 14/30 - Training Loss: 2.3234
Epoch 15/30 - Training Loss: 2.3234
Epoch 16/30 - Training Loss: 2.3234
Epoch 17/30 - Training Loss: 2.3234
Epoch 18/30 - Training Loss: 2.3234
Epoch 19/30 - Training Loss: 2.3234
Epoch 20/30 - Training Loss: 2.3234
Epoch 21/30 - Training Loss: 2.3234
Epoch 22/30 - Training Loss: 2.3234
Epoch 23/30 - Training Loss: 2.3234
Epoch 24/30 - Training Loss: 2.3234
Epoch 25/30 - Training Loss: 2.3234
Epoch 26/30 - Training Loss: 2.3234
Epoch 27/30 - Training Loss:

Activation function: linear
F1 score: 0.0000

Training Perceptron with sigmoid activation:
Epoch 1/30 - Training Loss: 20.7024
Epoch 2/30 - Training Loss: 20.7024
Epoch 3/30 - Training Loss: 20.7024
Epoch 4/30 - Training Loss: 20.7024
Epoch 5/30 - Training Loss: 20.7024
Epoch 6/30 - Training Loss: 20.7024
Epoch 7/30 - Training Loss: 20.7024
Epoch 8/30 - Training Loss: 20.7024
Epoch 9/30 - Training Loss: 20.7024
Epoch 10/30 - Training Loss: 20.7024
Epoch 11/30 - Training Loss: 20.7024
Epoch 12/30 - Training Loss: 20.7024
Epoch 13/30 - Training Loss: 20.7024
Epoch 14/30 - Training Loss: 20.7024
Epoch 15/30 - Training Loss: 20.7024
Epoch 16/30 - Training Loss: 20.7024
Epoch 17/30 - Training Loss: 20.7024
Epoch 18/30 - Training Loss: 20.7024
Epoch 19/30 - Training Loss: 20.7024
Epoch 20/30 - Training Loss: 20.7024
Epoch 21/30 - Training Loss: 20.7024
Epoch 22/30 - Training Loss: 20.7024
Epoch 23/30 - Training Loss: 20.7024
Epoch 24/30 - Training Loss: 20.7024
Epoch 25/30 - Training Loss: 2

Activation function: sigmoid
F1 score: 0.0955

Training Perceptron with relu activation:
Epoch 1/30 - Training Loss: 2.3234
Epoch 2/30 - Training Loss: 2.3234
Epoch 3/30 - Training Loss: 2.3234
Epoch 4/30 - Training Loss: 2.3234
Epoch 5/30 - Training Loss: 2.3234
Epoch 6/30 - Training Loss: 2.3234
Epoch 7/30 - Training Loss: 2.3234
Epoch 8/30 - Training Loss: 2.3234
Epoch 9/30 - Training Loss: 2.3234
Epoch 10/30 - Training Loss: 2.3234
Epoch 11/30 - Training Loss: 2.3234
Epoch 12/30 - Training Loss: 2.3234
Epoch 13/30 - Training Loss: 2.3234
Epoch 14/30 - Training Loss: 2.3234
Epoch 15/30 - Training Loss: 2.3234
Epoch 16/30 - Training Loss: 2.3234
Epoch 17/30 - Training Loss: 2.3234
Epoch 18/30 - Training Loss: 2.3234
Epoch 19/30 - Training Loss: 2.3234
Epoch 20/30 - Training Loss: 2.3234
Epoch 21/30 - Training Loss: 2.3234
Epoch 22/30 - Training Loss: 2.3234
Epoch 23/30 - Training Loss: 2.3234
Epoch 24/30 - Training Loss: 2.3234
Epoch 25/30 - Training Loss: 2.3234
Epoch 26/30 - Traini

Activation function: relu
F1 score: 0.4721


In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

def convert_to_binary(labels, digit):
    return np.array([1 if label == digit else 0 for label in labels])

class Perceptron:
    def __init__(self, input_size, activation_fn):
        self.weights = np.zeros(input_size)
        self.bias = 0
        self.activation_fn = activation_fn
        self.lr = 0.01

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def relu(self, x):
        return np.maximum(0, x)

    def linear(self, x):
        return x

    def activate(self, x):
        if self.activation_fn == 'sigmoid':
            return self.sigmoid(x)
        elif self.activation_fn == 'relu':
            return self.relu(x)
        elif self.activation_fn == 'linear':
            return self.linear(x)
        else:
            raise ValueError("Invalid activation function")

    def predict(self, X):
        return self.activate(np.dot(X, self.weights) + self.bias)

    def train(self, X, y, epochs, learning_rate):
        epsilon = 1e-10
        losses_train = []

        for epoch in range(epochs):
            for xi, yi in zip(X, y):
                prediction = self.predict(xi)
                prediction = np.clip(prediction, epsilon, 1 - epsilon)

                y_error = prediction - yi

                # Update weights and bias
                self.weights += self.lr * y_error * xi
                self.bias += self.lr * y_error

            # Calculate and record training loss
            loss_train = self.calculate_loss(X, y)
            losses_train.append(loss_train)

            # Print progress
            print(f"Epoch {epoch + 1}/{epochs} - Training Loss: {loss_train:.4f}")

        # Plot learning curves
        plt.plot(range(1, epochs + 1), losses_train, label='Training Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Cross-Entropy Loss')
        plt.title(f'Learning Curves ({self.activation_fn} activation)')
        plt.legend()
        plt.show()

    def calculate_loss(self, X, y):
        epsilon = 1e-10
        predictions = self.predict(X)
        predictions = np.clip(predictions, epsilon, 1 - epsilon)
        return -np.mean(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))

# Example usage
activation_functions = ['linear', 'sigmoid', 'relu']

for activation_function in activation_functions:
    print(f"\nTraining Perceptron with {activation_function} activation:")
    perceptron = Perceptron(input_size=X_train.shape[1], activation_fn=activation_function)
    perceptron.train(X_train, convert_to_binary(y_train, digit=5), epochs=30, learning_rate=0.01)

    # Evaluate on the test set
    predictions_test = perceptron.predict(X_test)
    predictions_test_binary = np.round(predictions_test)
    f1 = f1_score(convert_to_binary(y_test, digit=5), predictions_test_binary, average='macro')
    print(f"Activation function: {activation_function}")
    print(f"F1 score: {f1:.4f}")


Epoch 1/30 - Training Loss: 2.0422
Epoch 2/30 - Training Loss: 2.0833
Epoch 3/30 - Training Loss: 2.0579
Epoch 4/30 - Training Loss: 2.0272
Epoch 5/30 - Training Loss: 1.9851
Epoch 6/30 - Training Loss: 1.9261
Epoch 7/30 - Training Loss: 1.8314
Epoch 8/30 - Training Loss: 1.6225
Epoch 9/30 - Training Loss: 1.1963
Epoch 10/30 - Training Loss: 1.2027
Epoch 11/30 - Training Loss: 1.1938
Epoch 12/30 - Training Loss: 1.1829
Epoch 13/30 - Training Loss: 1.1719
Epoch 14/30 - Training Loss: 1.1637
Epoch 15/30 - Training Loss: 1.1577
Epoch 16/30 - Training Loss: 1.1688
Epoch 17/30 - Training Loss: 1.2253
Epoch 18/30 - Training Loss: 1.2233
Epoch 19/30 - Training Loss: 1.2194
Epoch 20/30 - Training Loss: 1.2120
Epoch 21/30 - Training Loss: 1.2019
Epoch 22/30 - Training Loss: 1.1886
Epoch 23/30 - Training Loss: 1.1748
Epoch 24/30 - Training Loss: 1.1620
Epoch 25/30 - Training Loss: 1.1464
Epoch 26/30 - Training Loss: 1.1352
Epoch 27/30 - Training Loss: 1.1080
Epoch 28/30 - Training Loss: 1.0906
E

ValueError: operands could not be broadcast together with shapes (1437,) (1437,10) 

In [18]:
from sklearn.preprocessing import LabelBinarizer

def convert_to_one_hot(labels):
    lb = LabelBinarizer()
    return lb.fit_transform(labels)

# Example usage
binary_labels_train = convert_to_one_hot(y_train)
nn_model.train(X_train, binary_labels_train, epochs=30, learning_rate=0.01)

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights and biases
        self.weights_hidden = np.random.randn(input_size, hidden_size)
        self.bias_hidden = np.zeros((1, hidden_size))
        self.weights_out = np.random.randn(hidden_size, output_size)
        self.bias_out = np.zeros((1, output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        self.hidden_layer_output = self.sigmoid(np.dot(X, self.weights_hidden) + self.bias_hidden)
        self.output = self.sigmoid(np.dot(self.hidden_layer_output, self.weights_out) + self.bias_out)

        return self.output

    def backward(self, X, y, learning_rate):
        output_error = y - self.output
        output_delta = output_error * self.sigmoid_derivative(self.output)

        hidden_layer_error = output_delta.dot(self.weights_out.T)
        hidden_layer_delta = hidden_layer_error * self.sigmoid_derivative(self.hidden_layer_output)

        # Update weights and biases
        self.weights_out += learning_rate * self.hidden_layer_output.T.dot(output_delta)
        self.bias_out += learning_rate * np.sum(output_delta, axis=0, keepdims=True)

        self.weights_hidden += learning_rate * X.T.dot(hidden_layer_delta)
        self.bias_hidden += learning_rate * np.sum(hidden_layer_delta, axis=0, keepdims=True)


    def train(self, X, y, epochs, learning_rate):
        losses_train = []

        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y, learning_rate)

            # Calculate and record training loss
            loss_train = self.calculate_loss(X, y)
            losses_train.append(loss_train)

            # Print progress
            print(f"Epoch {epoch + 1}/{epochs} - Training Loss: {loss_train:.4f}")

        # Plot learning curve
        plt.plot(range(1, epochs + 1), losses_train, label='Training Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Cross-Entropy Loss')
        plt.title('Neural Network Learning Curve')
        plt.legend()
        plt.show()

    def calculate_loss(self, X, y):
        predictions = self.forward(X)
        return -np.mean(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))

    def predict(self, X):
        return np.round(self.forward(X))


# Example usage
nn_model = NeuralNetwork(input_size=X_train.shape[1], hidden_size=64, output_size=len(np.unique(y_train)))
binary_labels_train = convert_to_binary(y_train, digit=5)
nn_model.train(X_train, binary_labels_train, epochs=30, learning_rate=0.01)

# Evaluate on the test set
predictions_test = nn_model.predict(X_test)
f1 = f1_score(convert_to_binary(y_test, digit=5), predictions_test, average='macro')
print(f"F1 score: {f1:.4f}")